https://www.cnblogs.com/pinard/p/6912636.html

https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.binom.html

$\Theta(g+1)=\underset{\theta}{\arg \max }\left(\int_{z} \log (p(X, Z \mid \theta)) p\left(Z \mid X, \Theta^{(g)}\right)\right) \mathrm{d} z$

In [1]:
# em 硬币 demo
# https://gitbook.cn/gitchat/column/5ebce48bb0115851722011b3/topic/5ec15f0f1a34fa16c60d9e1f
import numpy as np
from scipy.stats import binom


#一轮迭代处理
def single_iter(theta_priors, observations):
    """
    param observations：五组试验的观测值
    param theta_priors：上一轮迭代形成的 theta_A 和 theta_B
    """
    counts = {'A': {'H': 0, 'T': 0}, 'B': {'H': 0, 'T': 0}}
    theta_A = theta_priors[0]
    theta_B = theta_priors[1]

    #迭代计算每组试验的数据
    for observation in observations:
        len_observation = len(observation)
        num_heads = observation.sum()
        num_tails = len_observation-num_heads

        # 由上一步 获得 theta 的 先验概率
        # 计算 P(\theta)
        # 二项分布
        P_A = binom.pmf(num_heads, len_observation, theta_A) # 正面朝上次数， 观测次数， 上一步 A 硬币正面朝上的概率 ==》 硬币 A 正面朝上 出现次数为 num_heads 的概率
        P_B = binom.pmf(num_heads, len_observation, theta_B) # 正面朝上次数， 观测次数， 上一步 B 硬币正面朝上的概率 ==》 硬币 B 正面朝上 出现次数为 num_heads 的概率

        # 当 正面出现次数为 num_heads 时, 计算出硬币 A 和硬币 B 各自出现的概率
        weight_A = P_A / (P_A + P_B)
        weight_B = P_B / (P_A + P_B)

        # 更新在当前硬币 A 和硬币 B 各自出现的概率下，硬币 A 和硬币 B 各自的正反面次数
        # 计算 P(x|\theta)
        counts['A']['H'] += weight_A * num_heads # 正面
        counts['A']['T'] += weight_A * num_tails # 反面
        counts['B']['H'] += weight_B * num_heads # 正面
        counts['B']['T'] += weight_B * num_tails # 反面

    #重新估计新一轮的硬币 A 和硬币 B 正面向上的概率
    new_theta_A = counts['A']['H'] / (counts['A']['H'] + counts['A']['T'])
    new_theta_B = counts['B']['H'] / (counts['B']['H'] + counts['B']['T'])
    return [new_theta_A,new_theta_B]

In [2]:
observations = np.array([[1,0,0,0,1,1,0,1,0,1],
                            [1,1,1,1,0,1,1,1,0,1],
                            [1,0,1,1,1,1,1,0,1,1],
                            [1,0,1,0,0,0,1,1,0,0],
                            [0,1,1,1,0,1,1,1,0,1]])

prior = [0.6, 0.5] #设定初始的参数值
iteration = 0
iterations = 10000 #最多的迭代次数
tol = 1e-6         #判断参数收敛的阈值
while iteration < iterations:
    new_prior = single_iter(prior,observations)
    delta_change = np.abs(prior[0] - new_prior[0])
    if delta_change < tol:
        break
    else:
        prior = new_prior
        iteration += 1

print('迭代结束，总共迭代轮数{}'.format(iteration))
print('最终估计的参数{}'.format(new_prior))

迭代结束，总共迭代轮数36
最终估计的参数[0.72225028549926, 0.555438089938483]
